<a href="https://colab.research.google.com/github/khbae/trading/blob/master/12_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install twython

필요한 라이브러리 불러오기 및 감정사전 다운로드

In [0]:
import nltk
import pandas as pd
nltk.download("opinion_lexicon")
from nltk.corpus import opinion_lexicon
from nltk.tokenize import treebank

[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     /content/nltk_data...
[nltk_data]   Unzipping corpora/opinion_lexicon.zip.


google drive에서 데이터 불러오기

In [0]:
from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

file_id = '12VE5YW0-O2vc2DxoVYyxhCq3nv8rqwhh'

import io
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  _, done = downloader.next_chunk()

downloaded.seek(0)
print('Downloaded file contents are: {}'.format(downloaded.read()))

In [0]:
import os
os.listdir()

['datalab',
 '.local',
 '.cache',
 '.rnd',
 'article.csv',
 '.forever',
 'nltk_data',
 '.ipython',
 '.config']

**yahoo finance에서 스크랩한 csv파일 불러오기**



pd.read_csv("파일이름")=>working directory에서 csv파일을 읽어 python으로 가져오는 명령어. utf8은 문자열을 encoding하는 방식.

In [0]:
dfarticle=pd.read_csv("article.csv",encoding="utf8")

**<감정분석>
tokenize->token들을 감정사전에서 검색하여 positive/negative로 분류->positive token의 갯수와 negative token의 갯수를 비교하여 전체 기사 제목의 sentiment를 postive/negative/neutral로 분류**


In [0]:
sentiment=[]
for text in dfarticle["titles"]:

    tokenizer = treebank.TreebankWordTokenizer()
    #문장을 tokenize 할 수 있게 하는 객체 초기화
    pos_words = 0
    neg_words = 0
    tokenized_sent = [word.lower() for word in tokenizer.tokenize(text)]
    #print(tokenized_sent)를 해봄으로써 어떻게 tokenize되는지 확인
    
   

    for word in tokenized_sent:
        if word in opinion_lexicon.positive():
            pos_words += 1
        elif word in opinion_lexicon.negative():
            neg_words += 1
        else:
            pass

    if pos_words > neg_words:
        print('Positive')
        sentiment.append('Positive')
    elif pos_words < neg_words:
        print('Negative')
        sentiment.append('Negative')
    elif pos_words == neg_words:
        print('Neutral')
        sentiment.append('Neutral')

    

Neutral
Negative
Neutral
Positive
Neutral
Neutral
Neutral
Neutral
Positive
Negative
Negative
Neutral
Positive
Neutral
Neutral
Neutral
Neutral
Positive
Negative
Negative
Neutral
Positive
Neutral
Neutral
Neutral
Neutral
Positive
Negative
Negative
Neutral
Positive
Neutral
Neutral
Neutral
Neutral
Positive
Negative
Negative
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Positive
Negative
Negative
Neutral
Neutral
Neutral


**dfarticle에 분류된 sentiment 붙이기&csv파일로 저장**
data.to_csv("filename.csv")=>data를 csv파일로 working directory에 저장.

In [0]:
dfarticle["sentiment"]=sentiment
dfarticle.to_csv("article_sentiment.csv",encoding='utf8')

google drive에도 저장

In [0]:
from googleapiclient.http import MediaFileUpload


file_metadata = {
  'name': 'article_sentiment.csv',
  'mimeType': 'text/csv'
}
media = MediaFileUpload('article_sentiment.csv', 
                        mimetype='text/csv',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
print('File ID: {}'.format(created.get('id')))

File ID: 1cDacrlRoynmeAplcKYXnHnArN06J2yES
